# Final EDA and Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', 100)

In [2]:
#loading dataframes
df1=pd.read_excel("EDA_&_Modeling/Telco_customer_churn.xlsx")
df2=pd.read_excel("EDA_&_Modeling/Telco_customer_churn_status.xlsx")
df3=pd.read_excel("EDA_&_Modeling/Telco_customer_churn_services.xlsx")
df4=pd.read_excel("EDA_&_Modeling/Telco_customer_churn_demographics.xlsx")

In [3]:
#Mergin dataframes into one
df1_2=pd.merge(df1, df2, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)
df3_4=pd.merge(df3, df4, left_on='Customer ID', right_on='Customer ID', how='left')
customer_df=pd.merge(df1_2, df3_4, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)

In [4]:
#Cleaning up duplicates 
customer_df=customer_df.T.drop_duplicates().T
customer_df.columns=[column.lower().strip("_x").strip("_y").replace(" ","_")for column in customer_df.columns]  
customer_df["internet_services"]=customer_df["internet_service"].iloc[:,1:2]
customer_df["online_security"]=customer_df["online_securit"].iloc[:,1:2]
customer_df["online_backup_srvc"]=customer_df["online_backup"].iloc[:,1:2]
customer_df["multiple_line"]=customer_df["multiple_lines"].iloc[:,1:2]
customer_df["streaming_tv_srvc"]=customer_df["streaming_tv"].iloc[:,1:2]
customer_df["payment_methods"]=customer_df["payment_method"].iloc[:,:1]
customer_df["total_charge"]=customer_df["total_charges"].iloc[:,:1]
customer_df["contracts"]=customer_df["contract"].iloc[:,1:2]
customer_df["churn_reasons"]=customer_df["churn_reason"].iloc[:,1:2]
customer_df["streaming_movies_srvc"]=customer_df["streaming_movies"].iloc[:,1:2]

In [5]:
#Dropping Redundant columns
customer_df.drop(["total_charges","churn_score","churn_reason","lat_long","customerid","payment_method","contract","churn_label","count","countr","state","quarter","internet_service","online_securit","online_backup","multiple_lines","streaming_tv","streaming_movies"],axis=1,inplace=True)

In [6]:
#Cleaning missing values from "total_charge" column
hold=[]
for x in customer_df.index:
    if type(customer_df.total_charge[x])== str:
        hold.append(0)
    else:
        hold.append(customer_df.total_charge[x])
customer_df.total_charge=hold

In [7]:
#Binning age 
customer_df.age=pd.cut(x=customer_df['age'], bins=[10,20,30,40, 50,60,70,80], labels=["10","20","30","40", "50","60","70"])

In [8]:
#Filling NA values for churn reason
customer_df.churn_reasons=customer_df.churn_reasons.fillna("No reason given")

In [9]:
conditions=[
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==0),            
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==1),
]
choises=[
    "Not Churned",
    "No Reason",

]
customer_df.churn_categor=np.select(conditions,choises,default=customer_df.churn_categor)

In [10]:
 def to_binary(column):
    """
    Takes in a column and turns its value from a Yes or No to a binary. 
    """
    conditions=[
        (customer_df[column].str.lower()=='yes'),            
        (customer_df[column].str.lower()=='no'),
    ]
    choises=[
        1,
        0,

    ]
    customer_df[column]=np.select(conditions,choises,default=0)

In [11]:
#converting columns into binary
col_to_bin=["senior_citizen","device_protection_plan","partner","dependents","phone_service","paperless_billing","referred_a_friend","premium_tech_support","streaming_music","unlimited_data","under_30","online_security","multiple_line","online_backup_srvc","streaming_tv_srvc","streaming_movies_srvc","internet_services","tech_support","device_protection"]
for x in col_to_bin:
    to_binary(x)

In [12]:
customer_df=pd.get_dummies(customer_df,columns=["cit","customer_status","churn_categor","age","zip_code","gender","churn_reasons","offer","internet_type","contracts","payment_methods","number_of_dependents"],drop_first=True)

In [13]:
df=customer_df.astype("float")

In [14]:
#Splitting the data into X and Y
y=df["churn_value"]
X=df[df.columns[df.columns!="churn_value"]]

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2020, test_size=0.2)

In [16]:
params2={
    "max_depth":range(5,30,5),
    "min_samples_leaf":range(1,5,1),
    "min_samples_split":range(1,5,1),
    "bootstrap":["True","False"],
    "class_weight":["None","balanced"]  
}

In [17]:
RF=RandomForestClassifier(random_state=40)
grid_rf=GridSearchCV(RF, params2, cv=10, scoring="f1", verbose=1, n_jobs=-1)

In [18]:
grid_rf.fit(X_train,y_train)

Fitting 10 folds for each of 320 candidates, totalling 3200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 3200 out of 3200 | elapsed: 10.7min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=40),
             n_jobs=-1,
             param_grid={'bootstrap': ['True', 'False'],
                         'class_weight': ['None', 'balanced'],
                         'max_depth': range(5, 30, 5),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(1, 5)},
             scoring='f1', verbose=1)

In [19]:
print("Best Gridsearch Score: {}".format(grid_rf.best_score_))
print("Best Parameters: {}".format(grid_rf.best_params_))
print("Best Estimators: {}".format(grid_rf.best_estimator_))

Best Gridsearch Score: 1.0
Best Parameters: {'bootstrap': 'True', 'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Estimators: RandomForestClassifier(bootstrap='True', class_weight='balanced', max_depth=10,
                       random_state=40)


In [20]:
grid_rf_pred=grid_rf.best_estimator_.predict(X_test)

In [21]:
gs_acc2=metrics.recall_score(y_test, grid_rf_pred)
gs_rec2=metrics.accuracy_score(y_test, grid_rf_pred)
gs_f12=metrics.f1_score(y_test,grid_rf_pred)

In [22]:
print('GridSearch Logistic Regression Accuracy: {}'.format(gs_acc2))
print('GridSearch Logistic Regression Recall: {}'.format(gs_rec2))
print('GridSearch Logistic Regression F1: {}'.format(gs_f12))

GridSearch Logistic Regression Accuracy: 1.0
GridSearch Logistic Regression Recall: 1.0
GridSearch Logistic Regression F1: 1.0
